In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
import optuna
from functools import partial
import subprocess
import time

def run_java_anonymization(input_csv, output_csv, k_value, l_value=2, weights=None):
    """
    Führt das Java-Programm zur Anonymisierung mit den angegebenen Parametern aus.
    
    Args:
        input_csv (str): Pfad zur Eingabe-CSV-Datei.
        output_csv (str): Pfad zur Ausgabe-CSV-Datei.
        k_value (str): Wert für k-Anonymität.
        l_value (int, optional): Wert für l-Diversität (Standard: 2).
        weights (dict, optional): Gewichtungen für die Attribute, falls vorhanden.
    """
    # Startzeit messen
    start_time = time.time()

    # Pfade und Parameter
    java_file = "C:/Users/leesh/Desktop/BA/Notebooks/automated_anonymization/Anonymization/src/main/java/ARXAnonymization_w.java"
    class_file = "C:/Users/leesh/Desktop/BA/Notebooks/automated_anonymization/Anonymization/src/main/java/ARXAnonymization_w.class"
    libs_dir = "C:/Users/leesh/Desktop/BA/Notebooks/automated_anonymization/Anonymization/lib/*"

    # Gewichtungen als String formatieren, wenn vorhanden
    if weights is not None:
        weights_str = ','.join([str(weights[key]) for key in sorted(weights.keys())])
    else:
        weights_str = ""

    # Ausführung des Java-Programms
    print("weights_str", weights_str)
    run_command = [
        "java",
        "-cp", f"{libs_dir};C:/Users/leesh/Desktop/BA/Notebooks/automated_anonymization/Anonymization/src/main/java",
        "ARXAnonymization_w",
        input_csv,
        output_csv,
        str(k_value),
        str(l_value),
        weights_str
    ]


    print("Running Java code...")
    process = subprocess.Popen(run_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    # Lesen und Ausgeben der Ausgabe zeilenweise
    while True:
        output = process.stdout.readline()
        if output == "" and process.poll() is not None:
            break
        if output:
            print(output.strip())

    # Lesen und Ausgeben von Fehlern
    stderr_output = process.stderr.read()
    if stderr_output:
        print(f"Error:\n{stderr_output}")

    # Überprüfen des Rückgabecodes des Prozesses
    exit_code = process.poll()
    if exit_code == 0:
        print("Execution successful!")
    else:
        print(f"Execution failed with exit code {exit_code}")

    # Endzeit messen
    end_time = time.time()
    elapsed_time = end_time - start_time
    hours_total, rem_total = divmod(elapsed_time, 3600)
    minutes_total, seconds_total = divmod(rem_total, 60)
    print(f"Gesamte Ausführung hat {int(hours_total)} Stunden, {int(minutes_total)} Minuten, und {seconds_total:.2f} Sekunden gedauert.")
    
    return exit_code

def run_script_and_capture_output(command):
    """Führt ein Kommando aus und gibt die Ausgabe in Echtzeit zurück."""
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    while True:
        output = process.stdout.readline()
        if output == "" and process.poll() is not None:
            break
        if output:
            print(output.strip())

    # Lesen und Drucken von Fehlermeldungen
    stderr_output = process.stderr.read()
    if stderr_output:
        print(f"Error:\n{stderr_output}")

    return process.poll()  # Rückgabe des Exit-Codes

# Funktion zur Anonymisierung
def anonymize_data(df, weights, temp_input_csv="C:/Users/leesh/Desktop/BA/Notebooks/weight_optimization/temp_input.csv", temp_output_csv_template="C:/Users/leesh/Desktop/BA/Notebooks/weight_optimization/temp_output_k{}.csv"):
    # Speichern des DataFrames als CSV, damit das Java-Programm darauf zugreifen kann
    df.to_csv(temp_input_csv, index=False)

    k_values = [100]

    for k_value in k_values:
        temp_output_csv = temp_output_csv_template.format(k_value)
    
        print("Running Java anonymization with k={k_value}...")
        java_exit_code = run_java_anonymization(temp_input_csv, temp_output_csv, k_value, weights=weights)

        if java_exit_code != 0:
            print(f"Anonymization script failed with exit code {java_exit_code}.")
            continue

        # Warte auf das Erstellen der Datei
        for i in range(10):  # Versuche es 10 Mal mit jeweils 1 Sekunde Pause
            if os.path.exists(temp_output_csv):
                break
            time.sleep(1)
        else:
             # Wenn die Datei nicht gefunden wird, erstelle eine leere Datei
            print(f"{temp_output_csv} wurde nicht gefunden. Erstelle eine leere Datei.")
            pd.DataFrame().to_csv(temp_output_csv, index=False)  # Erstelle eine leere CSV
            raise FileNotFoundError(f"{temp_output_csv} wurde nicht gefunden.")
    
        # Laden der anonymisierten Daten zurück in einen DataFrame
        anonymized_df = pd.read_csv(temp_output_csv)
        print(f"Anonymization completed for k={k_value}. Output saved to {temp_output_csv}.")

    print("All anonymization processes completed.")
    return anonymized_df


# Processing-Funktion für Intervalle und Mengen
def process_value(value):
    if isinstance(value, str):
        value = value.strip()
        if value.startswith('[') and value.endswith('['):
            try:
                start, end = map(float, value[1:-1].split(','))
                return (start + end) / 2
            except ValueError:
                return np.nan
        elif value.startswith('{') and value.endswith('}'):
            try:
                values = list(map(float, value[1:-1].split(',')))
                return np.mean(values)
            except ValueError:
                return np.nan
        else:
            try:
                return float(value)
            except ValueError:
                return np.nan
    return value


# Funktion zur Bewertung der Modellleistung
def evaluate_model(df, weights):
    df_anonymized = anonymize_data(df, weights)
    if df_anonymized is None:
        return 0  # Rückgabe von 0, wenn die Anonymisierung fehlschlägt

    # Sicherstellen, dass keine ungültigen Werte vorhanden sind
    df_anonymized = df_anonymized.drop(columns=["id"])
    df_anonymized = df_anonymized[~df_anonymized.isin(['*']).any(axis=1)]
    df_anonymized = df_anonymized.dropna()
    
    X = df_anonymized.drop(columns=["cardio"])
    y = df_anonymized["cardio"]

    for column in X.columns:
        X[column] = X[column].apply(process_value)

    #X.to_csv("C:/Users/leesh/Desktop/BA/Notebooks/weight_optimization/temp_output_modified.csv", index=False)
    
    clf = RandomForestClassifier()
    scores = cross_val_score(clf, X, y, cv=5, scoring='accuracy')
    return scores.mean()

# Optuna-Ziel-Funktion für die Optimierung
def objective(trial, df):
    
    # Bayesian Optimization durch Variieren der Gewichtungen
    weights = {
        "age": trial.suggest_float("age_weight", 0.0, 1.0),
        "gender": trial.suggest_float("gender_weight", 0.0, 1.0),
        "height": trial.suggest_float("height_weight", 0.0, 1.0),
        "weight": trial.suggest_float("weight_weight", 0.0, 1.0),
        "ap_hi": trial.suggest_float("ap_hi_weight", 0.0, 1.0),
        "ap_lo": trial.suggest_float("ap_lo_weight", 0.0, 1.0),
        "cholesterol": trial.suggest_float("cholesterol_weight", 0.0, 1.0),
        "gluc": trial.suggest_float("gluc_weight", 0.0, 1.0),
        "smoke": trial.suggest_float("smoke_weight", 0.0, 1.0),
        "alco": trial.suggest_float("alco_weight", 0.0, 1.0),
        "active": trial.suggest_float("active_weight", 0.0, 1.0),
    }

    score = evaluate_model(df, weights)
    return score

# Funktion zum Laden von CSV-Dateien und Optimierung pro Datei
def optimize_weights_for_datasets(input_folder, output_csv):
    # Prüfen, ob es eine bestehende Ausgabe gibt, um Fortschritt fortzusetzen
    if os.path.exists(output_csv):
        results_df = pd.read_csv(output_csv)
    else:
        results_df = pd.DataFrame(columns=["dataset", "trial_number", "age_weight", "gender_weight", "height_weight", 
                                           "weight_weight", "ap_hi_weight", "ap_lo_weight", "cholesterol_weight", 
                                           "gluc_weight", "smoke_weight", "alco_weight", "active_weight", "score"])

    csv_files = [f for f in os.listdir(input_folder) if f.endswith('.csv')]

    for csv_file in csv_files:
        dataset_name = csv_file.split('.')[0]
        df = pd.read_csv(os.path.join(input_folder, csv_file), delimiter = ';')
        df['weight'] = df['weight'].str.replace(',', '.').apply(
            lambda x: float(x) if x != '*' else x
        )

        # Prüfen, ob für diesen Datensatz schon Optimierungen gemacht wurden
        existing_trials = results_df[results_df["dataset"] == dataset_name]
        if not existing_trials.empty:
            last_trial_number = existing_trials["trial_number"].max()
        else:
            last_trial_number = 0

        # Bayesian Optimization Setup
        study = optuna.create_study(direction="maximize")
        study.optimize(partial(objective, df=df), n_trials=50, n_jobs=1)

        # Ergebnisse speichern
        for i, trial in enumerate(study.trials):
            if i + 1 > last_trial_number:  # Nur neue Ergebnisse speichern
                result = {
                    "dataset": dataset_name,
                    "trial_number": i + 1,
                    "age_weight": trial.params["age_weight"],
                    "gender_weight": trial.params["gender_weight"],
                    "height_weight": trial.params["height_weight"],
                    "weight_weight": trial.params["weight_weight"],
                    "ap_hi_weight": trial.params["ap_hi_weight"],
                    "ap_lo_weight": trial.params["ap_lo_weight"],
                    "cholesterol_weight": trial.params["cholesterol_weight"],
                    "gluc_weight": trial.params["gluc_weight"],
                    "smoke_weight": trial.params["smoke_weight"],
                    "alco_weight": trial.params["alco_weight"],
                    "active_weight": trial.params["active_weight"],
                    "score": trial.value
                }
                #new_row = pd.DataFrame([result])
                #results_df = pd.concat([results_df, new_row], ignore_index=True)
                results_df = pd.concat([results_df, pd.DataFrame([result])], ignore_index=True)
                results_df.to_csv(output_csv, index=False)

# Ausführen der Optimierung für Datensätze in einem Ordner
input_folder = "C:/Users/leesh/Desktop/BA/Notebooks/weight_optimization"  # Ordner, der die CSV-Dateien enthält
output_csv = "C:/Users/leesh/Desktop/BA/Notebooks/weight_optimization/weight_optimization_results.csv"  # Datei, in der die Ergebnisse gespeichert werden
optimize_weights_for_datasets(input_folder, output_csv)


[I 2024-09-17 00:45:19,376] A new study created in memory with name: no-name-3c9eb8f4-3693-4168-a44b-6070f5e035d4


Running Java anonymization with k={k_value}...
weights_str 0.21574287416706084,0.40957365069666696,0.3574217188032208,0.15208455495165651,0.2044144578432986,0.2451686388247012,0.9857278317123906,0.33505457019162976,0.3798096911450387,0.44920441438273817,0.11505688692359883
Running Java code...
Anonymization successful for k = 100, l = 2
Execution successful!
Gesamte Ausführung hat 0 Stunden, 0 Minuten, und 33.14 Sekunden gedauert.
Anonymization completed for k=100. Output saved to C:/Users/leesh/Desktop/BA/Notebooks/weight_optimization/temp_output_k100.csv.
All anonymization processes completed.


[I 2024-09-17 00:46:00,788] Trial 0 finished with value: 0.6121805662595639 and parameters: {'age_weight': 0.40957365069666696, 'gender_weight': 0.9857278317123906, 'height_weight': 0.3798096911450387, 'weight_weight': 0.11505688692359883, 'ap_hi_weight': 0.15208455495165651, 'ap_lo_weight': 0.2044144578432986, 'cholesterol_weight': 0.2451686388247012, 'gluc_weight': 0.33505457019162976, 'smoke_weight': 0.44920441438273817, 'alco_weight': 0.3574217188032208, 'active_weight': 0.21574287416706084}. Best is trial 0 with value: 0.6121805662595639.


Running Java anonymization with k={k_value}...
weights_str 0.1359381631939075,0.5101456315067897,0.4699407046328934,0.6856227677982788,0.5192125246510133,0.025572354290508637,0.1531621072076197,0.6990852524905248,0.11311873890415647,0.42581295499661664,0.4716266383578126
Running Java code...
Anonymization successful for k = 100, l = 2
Execution successful!
Gesamte Ausführung hat 0 Stunden, 0 Minuten, und 32.79 Sekunden gedauert.
Anonymization completed for k=100. Output saved to C:/Users/leesh/Desktop/BA/Notebooks/weight_optimization/temp_output_k100.csv.
All anonymization processes completed.


[I 2024-09-17 00:46:42,256] Trial 1 finished with value: 0.713108573705543 and parameters: {'age_weight': 0.5101456315067897, 'gender_weight': 0.1531621072076197, 'height_weight': 0.11311873890415647, 'weight_weight': 0.4716266383578126, 'ap_hi_weight': 0.6856227677982788, 'ap_lo_weight': 0.5192125246510133, 'cholesterol_weight': 0.025572354290508637, 'gluc_weight': 0.6990852524905248, 'smoke_weight': 0.42581295499661664, 'alco_weight': 0.4699407046328934, 'active_weight': 0.1359381631939075}. Best is trial 1 with value: 0.713108573705543.


...

Running Java anonymization with k={k_value}...
weights_str 0.23217277270663755,0.10091960140266337,0.3434541236251434,0.1809224633858381,0.8784468792953535,0.879318119416177,0.2847418147426821,0.0491782583486553,0.07268948027236372,0.36001477658118075,0.6231757166495313
Running Java code...
Anonymization successful for k = 100, l = 2
Execution successful!
Gesamte Ausführung hat 0 Stunden, 0 Minuten, und 33.57 Sekunden gedauert.
Anonymization completed for k=100. Output saved to C:/Users/leesh/Desktop/BA/Notebooks/weight_optimization/temp_output_k100.csv.
All anonymization processes completed.


[I 2024-09-17 01:20:42,800] Trial 48 finished with value: 0.7130547886082899 and parameters: {'age_weight': 0.10091960140266337, 'gender_weight': 0.2847418147426821, 'height_weight': 0.07268948027236372, 'weight_weight': 0.6231757166495313, 'ap_hi_weight': 0.1809224633858381, 'ap_lo_weight': 0.8784468792953535, 'cholesterol_weight': 0.879318119416177, 'gluc_weight': 0.0491782583486553, 'smoke_weight': 0.36001477658118075, 'alco_weight': 0.3434541236251434, 'active_weight': 0.23217277270663755}. Best is trial 46 with value: 0.7231185209624819.


Running Java anonymization with k={k_value}...
weights_str 0.10177523620264184,0.2570750488900398,0.5054829604954933,0.005540229996227941,0.7142765675680814,0.4095220850382716,0.7479279826848223,0.5888181056533438,0.5501810034219788,0.2823727861204047,0.7439590294256764
Running Java code...
Anonymization successful for k = 100, l = 2
Execution successful!
Gesamte Ausführung hat 0 Stunden, 0 Minuten, und 32.83 Sekunden gedauert.
Anonymization completed for k=100. Output saved to C:/Users/leesh/Desktop/BA/Notebooks/weight_optimization/temp_output_k100.csv.
All anonymization processes completed.


[I 2024-09-17 01:21:25,027] Trial 49 finished with value: 0.7132211145612029 and parameters: {'age_weight': 0.2570750488900398, 'gender_weight': 0.7479279826848223, 'height_weight': 0.5501810034219788, 'weight_weight': 0.7439590294256764, 'ap_hi_weight': 0.005540229996227941, 'ap_lo_weight': 0.7142765675680814, 'cholesterol_weight': 0.4095220850382716, 'gluc_weight': 0.5888181056533438, 'smoke_weight': 0.2823727861204047, 'alco_weight': 0.5054829604954933, 'active_weight': 0.10177523620264184}. Best is trial 46 with value: 0.7231185209624819.
C:\Users\leesh\AppData\Local\Temp\ipykernel_197716\770707487.py:255: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, pd.DataFrame([result])], ignore_index=True)


ValueError: could not convert string to float: '[40. 120['